# KAIST AI605 Assignment 3: Transformer


## Environment
You will only use Python 3.7 and PyTorch 1.9, which is already available on Colab:

In [ ]:
from platform import python_version
import torch

print("python", python_version())
print("torch", torch.__version__)

python 3.7.10
torch 1.8.1+cu101


## 1. Attention Layer

We will first start with going over a few concepts that you learned in your high school statistics class.
The variance of a random variable $X$, $\text{Var}(X)$ is defined as $\text{E}[(X-\mu)^2]$ where $\mu$ is the mean of $X$.
Furthermore, given two independent random variables $X$ and $Y$ and a constant $a$,
$$ \text{Var}(X+Y) = \text{Var}(X) + \text{Var}(Y),$$
$$ \text{Var}(aX) = a^2\text{Var}(X),$$
$$ \text{Var}(XY) = \text{E}(X^2)\text{E}(Y^2) - [\text{E}(X)]^2[\text{E}(Y)]^2.$$

> **Problem 1.1** *(3 points)*
  Suppose we are given two sets of $n$ random variables, $X_1 \dots X_n$ and $Y_1 \dots Y_n$,
  where all of these $2n$ variables are mutually independent and have a mean of $0$ and a variance of $1$.
  Prove that
  $$\text{Var}\left(\sum_i^n X_i Y_i\right) = n.$$

In Lecture 11 and 12, we discussed how the attention is computed in Transformer via the following equation,
  $$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V.$$

> **Problem 1.2** *(3 points)* 
  Suppose $Q$ and $K$ are matrices of independent variables each of which has a mean of $0$ and a variance of $1$.
  Using what you learned from Problem 1.1., show that
  $$\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) = 1.$$

> **Problem 1.3** *(4 points)*
  What would happen if the assumption that the variance of $Q$ and $K$ is $1$ does not hold?
  Consider each case of it being higher and lower than $1$ and conjecture what it implies, respectively.

## 2. Transformer

In this section, you will implement Transformer for a few tasks that are simpler than machine translation.
First, go through [Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html) and make sure you understand every block of the code.
Then, you will reuse these code where appropriate to create models for following three tasks.
Note that we do not provide a separate training or evaluation data, so it is your job to be able to create these in a reasonable manner.

> **Problem 2.1** *(4 points)*
  Create a model that takes a random set of input symbols from a vocabulary of digits (i.e. 0, 1, ... , 8, 9) as the input and generate back the same symbols.
  Instead of varying length, we fix the length to 32.
  Make sure to report that your model's accuracy (gives credit only if the entire output sequence is correct) goes above 90%.
  Note that a similar problem is also in Annotated Transformer, and copying code is allowed.


> **Problem 2.2** *(6 points)*
  Now, we will implement a bit more useful function, so-called spelling error correction.
  Your job is to create a model whose input is a word with spelling errors, and the output is the spelling-corrected word.
  Here, your vocabulary will be character instead of word.
  You can create your own training data by using an existing text corpus as the target and inject noise into it to use it as the input.
  You are free to use whichever text corpus you like.
  If you can't think of one, please use context data in SQuAD Dataset (see Assignment 2).
  Report accuracy in your own evaluation data (you will receive full credit as long as both the evaluation data and the accuracy are reasonable),
  and also show 5 examples where it succeeds at correcting spelling.

> **Problem 2.3 (bonus)** *(3 points)*
  Extend this word-level spelling correction model to sentence-level.
  You do not have to report accuracy, but find 3 examples where the word-level model fails and sentence-level model correctly predicts.